# <span style="color:blue"> *Torneio Alfabeta Kalah*</span>
## Introdução à Inteligência Artificial (2023-24)
### 3º Projecto

<img src="Imagens/uiuimancala.png" alt="Drawing" style="width: 200px;"/>

## O Kalah

É quase certo que a maior parte de voçês já ouviu falar ou jogou o [Kalah](https://en.wikipedia.org/wiki/Kalah), que é uma variante moderna da família de jogos Mancala, criado em 1940 por William Champion, mas com origens muito antigas e que aparece numa explosão de ludo-diversidade em vários continentes e culturas.

<img src="Imagens/mancalaAreia.jpg" alt="Drawing" style="width: 200px;"/>


### Como funciona o jogo? 

<img src="Imagens/kalahBoard.png" alt="Drawing" style="width: 400px;"/>


#### Materiais:
O Mancala (a.k.a. Kalah) é jogado por dois jogadores: o Sul e o Norte, que jogam alternadamente num tabuleiro rectangular (ver a figura de cima) com 6 poços para cada jogador e um poço maior ou armazém que se chama de kalah e que fica à direita de cada jogador. As etiquetas dos poços (carregadas com barras se forem do jogador NORTE) correspondem à distância no sentido horário a que se encontram do kalah do jogador respectivo.


#### Tabuleiro inicial
No tabuleiro inicial standard, cada jogador começa com 4 peças (a.k.a sementes) em cada um dos seus 6 poços e ambos os kalahs vazios.

#### Jogada
Quando chega a sua vez, o jogador selecciona um dos poços com sementes, remove-as desse poço e começa a redistribuí-las (semeá-las) no sentido anti-horário, deixando uma semente por poço (seu ou do seu adversário) e pelo respectivo kalah, saltando o kalah do adversário. Se a última semente cair no seu kalah, volta a jogar, senão é a vez do adversário.

Se a última semente cair no seu lado num poço vazio então captura esse peça e as do poço do adversário que se encontra mesmo em frente, depositando-as no seu kalah.

Os kalahs são armazéns de sementes das quais as sementes nunca mais saiem e serão importantes para determinar o vencedor.


#### Fim do jogo
O jogo acaba quando, no final de uma jogada, o jogador não tiver mais sementes nos seus 6 poços. Nesse caso, o adversário captura todas as sementes do seu lado, depositando-as no seu kalah.

#### O vencedor
Após o final do jogo, o jogador que tenha mais peças no respectivo kalah ganha. Pode acontecer um empate se ambos acabarem com 24 sementes.

Eis um [video com a apresentação das regras](http://modelai.gettysburg.edu/2022/fairkalah/mancala-rules.mp4) (5:52) e  [um jogo completo de demonstração](http://modelai.gettysburg.edu/2022/fairkalah/mancala-game.mp4)(6:20)

### O Kalah Injusto
A partir do tabuleiro inicial standard é possível a jogada perfeita, fazendo com que o jogador SUL que abre o jogo ganhe 29 a 19 mesmo que o jogador NORTE jogue também de forma óptima, resistindo o melhor possível. Assim o kalah é um jogo "injusto", dando vantagem ao primeiro jogador.

### O Kalah Justo
Todd Neller e Taylor Neller encontraram um conjunto de 254 tabuleiros iniciais em que o jogo perfeito resulta num empate. A essa variante do jogo que começa com tabuleiros iniciais justos chamaram de KalahJusto.
Temos o conjunto de 254 tabuleiros no ficheiro anexo `fair-kalahs.py`.

## Objectivos do projecto
Pretende-se que dada uma formulação e implementação do jogo, os grupos de alunos:

1. Criem um jogador, que corresponde a uma função de avaliação a ser usada pelo algoritmo alfa-beta para qualquer profundidade, par ou ímpar. Aconselhamos que adoptem o paradigma "anytime", i.e. que desenvolvam várias versões do jogador, começando com um jogador simples sendo progressivamente melhorado, de modo a que tenham um jogador disponível na data limite. Desenvolvam e comparem o desempenho de vários jogadores, para diferentes limites de profundidade e depois seleccionem o melhor deles.
<br><br>
O jogador seleccionado será utilizado num campeonato, de todos contra todos, i.e. cada jogador do grupo irá jogar vários jogos contra todos os dos outros grupos cada nível de profundidade. Nesse campeonato, iremos seleccionar um dos tabuleiros justos ao acaso e todo os jogos de cada torneio (um torneio para um limite de profundidade diferente) começarão com esse tabuleiro justo. Nós próprios entraremos no campeonato com um jogador simples, o <span style="color:blue"> ***Chapiteau***. </span>

<img src="Imagens\theStrongestMan.gif" alt="Drawing" style="width: 200px;"/>

<span style="color:blue"> **Chapiteau**. </span>: tenta maximizar a diferença entre o número das suas sementes (no 6 poços e armazém) e as do seu adversário quando o jogo não é final. Quando o jogo é final, +100 para a vitória, -100 para a derrota e 0 para o empate.

## Formulação do Kalah em pyhton

<img src="Imagens\python.gif" alt="Drawing" style="width: 150px;"/>

Vamos descrever de modo sumário como está modelizado o jogo do Kalah, através das classes `KalahState` e `Kalah` que estão em `kalah.py`.

### O estado do jogo
Na classe `KalahState` temos três atributos principais:

* `to_move`: que indica quem é o próximo a jogar (0 se for o SUL e 1 se for o NORTE)
* `pass_turn`: com um valor booleano que indica se o próximo jogador passa ou não. 
* `state`: que é uma lista com 14 elementos, com a sequência de números, correspondendo ao número de sementes nos poços e kalahs, pela ordem anti-horária a começar pelo fundo à esquerda: a posição 0 da lista representa o poço 6 do SUL, a posição 6 corresponde ao kalah do SUL, a posição 7 corresponde ao poço 1 do NORTE e a posição 13 será o kalah do NORTE.

Fazemos um parêntesis para apresentar as acções possíveis.

### (As acções)
As acções, são na verdade representadas pelo indíce da lista (state) correspondente ao poço que se quer pegar ou então -1 se for 'passo' porque o adversário volta a jogar. O jogador SUL só poderá jogar as acções em {-1,0,1,2,3,4,5} e o jogador NORTE as acções de {-1,7,8,9,10,11,12}.

Regressemos à implementação do estado.

#### O construtor 
...recebe no máximo 2 argumentos. O `fairkalah_state_index` para indicar se queremos jogar (-1) com o tabuleiro standard, com um dos tabuleiros justos (=0) escolhido ao acaso ou com um determinado tabuleiro justo (>0); o `other` que indica se queremos começar por um tabuleiro específico que passamos como um tuplo $(proximo\_a\_jogar,listaSementes)$. Em qualquer caso o atributo `pass_turn` começa a False.

Vejemos alguns exemplos, após importarmos `kalah.py` o qual por sua vez importa todos os kalahs justos de `fairKalahs.py`, em que vamos fazer uso da versão string do estado definida em `__str__`.

In [1]:
from kalah import *

Criemos o tabuleiro inicial standard:

In [2]:
mx=KalahState()
mx

     _  _  _  _  _  _
     1  2  3  4  5  6
-------------------------
|  | 4| 4| 4| 4| 4| 4|  |
| 0|-----------------| 0|
|  | 4| 4| 4| 4| 4| 4|  | <--
-------------------------
     6  5  4  3  2  1

Geremos um tabuleiro inicial justo, por exemplo o 56:

In [3]:
mx=KalahState(56)
mx

     _  _  _  _  _  _
     1  2  3  4  5  6
-------------------------
|  | 4| 4| 4| 3| 4| 4|  |
| 0|-----------------| 0|
|  | 3| 5| 4| 4| 4| 5|  | <--
-------------------------
     6  5  4  3  2  1

Podemos também gerar um dos justo ao acaso (podem disparar o run várias vezes):

In [22]:
mx=KalahState(0)
mx

     _  _  _  _  _  _
     1  2  3  4  5  6
-------------------------
|  | 4| 5| 4| 3| 4| 4|  |
| 0|-----------------| 0|
|  | 3| 5| 4| 4| 4| 4|  | <--
-------------------------
     6  5  4  3  2  1

Ou podemos gerar um tabuleiro explicitamente, que até pode ser um tabuleiro impossível de acontecer num jogo mas que pode servir para testarem a função de avaliação:

In [5]:
mx=KalahState(other=(0,[6,5,4,3,2,1,0,4,4,4,4,4,4,3]))
mx

     _  _  _  _  _  _
     1  2  3  4  5  6
-------------------------
|  | 4| 4| 4| 4| 4| 4|  |
| 3|-----------------| 0|
|  | 6| 5| 4| 3| 2| 1|  | <--
-------------------------
     6  5  4  3  2  1

#### Os métodos de KalahState
Eis o que fazem as funções mais importantes:

* `get_legal_moves` que devolve todas as acções válidas ordenadas por ordem crescente.
* `is_game_over` que verifica se o jogo chegou ao fim.
* `result` que determina o resultado de um jogo finalizado: +1 se ganha o SUL, -1 se ganha o NORTE ou 0 em caso de empate.
* `real_move` que gera um novo estado com base na acção.

### A classe `Kalah` que representa um Jogo
A classe `Kalah` (em `Kalah.py`) é a subclasse de `Game`, que recebe os mesmos 2 argumentos que `KalahState`, gerando a instância desta última classe para inicializar o seu atributo `initial`, onde se guarda a representação do tabuleiro inicial e do jogador que começa, bem como a indicação que o primeiro lance não será uma 'passagem' de vez.

Teremos os métodos habituais:
* `actions`: que devolve a lista de acções para um determinado estado.
* `result`: que devolve o estado que resulta da aplicação de uma acção a um outro estado.
* `terminal_test`: que verifica se o jogo acabou ou não.
* `utility`: que devolve +1 se ganhou o SUL, -1 se ganhou o NORTE e 0 em caso de empate.
* `display`: que apresenta em modo o texto o estado do jogo.

### Definição de jogadores
Vamos ter uma classe para criar diferentes jogadores, indicando o seu nome e a função que utilizam para decidir a jogada e tendo um método `display` associado.

In [6]:
class Jogador():
    def __init__(self, nome, fun):
        self.nome = nome
        self.fun = fun
    def display(self):
        print(self.nome+" ")
        

#### O jogador aleatório
<img src="Imagens/diceArt.gif" alt="Drawing" style="width: 200px;"/>

Eis a classe dos jogadores aleatórios, que escolhem uma das acções ao acaso.

In [7]:
class JogadorAleat(Jogador):
    def __init__(self, nome):
        self.nome = nome
        self.fun = lambda game, state: random.choice(game.actions(state)) 

### Função que realiza um jogo entre 2 jogadores
Eis uma função que recebe uma instância de `Game`, dois jogadores e a flag `verbose` para indicar se queremos ver ou não o traço do jogo em modo texto (a False por omissão). Devolve uma lista com o par de jogadores que jogou, a lista ordenada de jogadas e o resultado final, mostrando o nome do jogador vencedor ou 'Empate' em caso de empate. 

In [8]:
##########  para ser independente dos jogos deveria devolver um método em string ou um atributo
def joga11(game, jog1, jog2,verbose=False):
    ### jog1 e jog2 são jogadores com funções que dado um estado do jogo devolvem a jogada que escolheram
    ### devolve o par de jogadores, a lista de jogadas e o resultado
    estado=game.initial
    proxjog = jog1
    lista_jogadas=[]
    lance = 0
    while not game.terminal_test(estado):
        if verbose:
            print('----------   LANCE:',lance)
            game.display(estado)
        jogada = proxjog.fun(game, estado)
        if verbose:
            print('JOGADA=',jogada)
        estado=game.result(estado,jogada)
        lista_jogadas.append(jogada)
        proxjog = jog2 if proxjog == jog1 else jog1
        lance+=1
    #p jogou e ganhou
    util=game.utility(estado,0)
    if util == 1:
        resultado=jog1.nome
    elif util== -1:
        resultado = jog2.nome
    else:
        resultado='Empate'
    return ((jog1.nome,jog2.nome),lista_jogadas,resultado)

Vamos realizar um jogo entre dois jogadores aleatórios: El Caos e La Entropia, com o modo `verbose` a False. Podem colocá-lo a True se qusierem ver a evolução do jogo.

In [9]:
el_caos=JogadorAleat('El Caos')
la_entropia=JogadorAleat('La Entropia')
mx = Kalah(20)
joga11(mx, el_caos, la_entropia)

(('El Caos', 'La Entropia'),
 [4,
  10,
  5,
  10,
  2,
  -1,
  1,
  11,
  2,
  9,
  -1,
  10,
  4,
  11,
  -1,
  7,
  3,
  10,
  1,
  9,
  0,
  -1,
  4,
  -1,
  2,
  10,
  3,
  8,
  5,
  7,
  4,
  10,
  1,
  8,
  3,
  11,
  5,
  -1,
  1,
  9,
  -1,
  12,
  2,
  10,
  3,
  -1,
  5,
  7,
  4,
  8,
  0,
  9,
  -1,
  7,
  5,
  -1,
  1,
  12,
  3,
  11,
  0,
  10,
  2,
  11,
  3,
  12,
  5,
  -1,
  0,
  7],
 'El Caos')

### O Jogador Alfabeta Minimax
Vamos criar a subclasse de `jogador` para usarmos com todos os jogadores que usam o minimax alfabeta com profundidade limitada (utiliza a função `alphabeta_cutoff_search_new` definida em `jogos.py`). 

In [10]:
class JogadorAlfaBeta(Jogador):
    def __init__(self, nome, depth,fun_eval):
        self.nome = nome
        self.fun = lambda game, state: alphabeta_cutoff_search_new(state,game,depth,eval_fn=fun_eval)

### Jogador El Caos Inteligente
Vamos definir a função de avaliação de um jogador que, se vir o fim do jogo, avalia-o bem: como derrota (-100), vitória (+100) ou empate (0). Devolve também 0 no caso de estar perante um tabuleiro não terminal. É aleatório porque a função 
`alphabeta_cutoff_search_new`baralha aleatoriamente as jogadas do jogador que está a decidir a jogada.

Comecemos pela função de avaliação.

In [11]:
def f_caos_intel(estado,jogador):
    """Quando é terminal: +100 para vitória, -100 para a derrota e 0 para o empate.
       Quando o tabuleiro é não terminal devolve 0, o que quer dizer que como o minimax baralha as acções, será random"""
    if estado.is_game_over():
        aux = estado.result()
        return aux*100 if jogador == estado.SOUTH else aux*-100
    return 0

Criemos o jogador para uma profundidade 6:

In [12]:
el_caos_int6=JogadorAlfaBeta("El Caos Inteligente 6",6,f_caos_intel)

Joguemos um jogo kalah justo (neste caso, o de índice 20) entre estas duas variantes do jogador aleatório e mostremos o resultado (repitam, porque há muita aleatoriedade nestes jogadores).

In [13]:
el_caos=JogadorAleat('El Caos')
mx = Kalah(20)
_,_,res=joga11(mx, el_caos_int6,el_caos)
res

'El Caos Inteligente 6'

### N Pares de jogos
Vejamos uma função que realiza $N$ pares de jogos entre dois jogadores, $N$ jogos como SUL e $N$ como NORTE. Devolve um tuplo com 4 elementos: o número de vitórias de cada um e de empates quando o primeiro é SUL e o segundo NORTE, o nº de vitórias de cada um e de empates quando os papeís se invertem, o nº total de vitórias de cada um e de empates e finalmente o score de cada um. A pontuação depende da tabela de `scores`, que neste caso indica que a vitória vale $3$, a derrota $0$ e o empate $1$. É a mesma escala que iremos utilizar no torneio.

In [14]:
scores={'Vitoria': 3, 'Empate': 1}

def traduzPontos(tabela):
    tabelaScore={}
    empates=tabela['Empate']
    for x in tabela:
        if x != 'Empate':
            tabelaScore[x]=scores['Vitoria']*tabela[x]+empates
    return tabelaScore

def jogaNpares(jogo,n,jog1,jog2):
    tabelaPrim={jog1.nome:0, jog2.nome:0, 'Empate':0}
    tabelaSeg={jog1.nome:0, jog2.nome:0, 'Empate':0}
    tabela={}
    for _ in range(n):
        _,_,vencedor=joga11(jogo,jog1,jog2)
        tabelaPrim[vencedor]+=1
        _,_,vencedor=joga11(jogo,jog2,jog1)
        tabelaSeg[vencedor]+=1
    for x in tabelaPrim:
        tabela[x]=tabelaPrim[x]+tabelaSeg[x]
    return tabelaPrim,tabelaSeg,tabela,traduzPontos(tabela)

Façamos 300 pares de jogos entre o El Caos e o El Caos Inteligente 6 e vejemos a pontuação final para confirmar que a capacidade de procurar a vitória e evitar a derrota compensa, mesmo quando se joga aleatoriamente quando no horizonte não se vislumbram os estados finais. Usemos o kalah justo com nº 10

In [15]:
jogo=Kalah(10)
jogaNpares(jogo,300,el_caos,el_caos_int6)

({'El Caos': 88, 'El Caos Inteligente 6': 193, 'Empate': 19},
 {'El Caos': 124, 'El Caos Inteligente 6': 160, 'Empate': 16},
 {'El Caos': 212, 'El Caos Inteligente 6': 353, 'Empate': 35},
 {'El Caos': 671, 'El Caos Inteligente 6': 1094})

### Função para Torneio entre vários jogadores
Eis a função que realiza um torneio entre vários jogadores. Queremos que cada um realize um número $N$ de jogos contra todos os outros como SUL, para um dado Kalah justo e a uma certa profundidade. Na prática, todos realizam um nº $N$ de jogos contra todos os outros, tanto no papel de 'SUL' como no papel de 'NORTE'.

In [16]:
def incorpora(tabela,tx):
    for jog in tx:
        if jog not in tabela:
            tabela[jog]=tx[jog]
        else:
            tabela[jog]+=tx[jog]
    
def torneio(n,jogadores,jogo_id=0):
    jogo=Kalah(jogo_id)  # jogo gerado ao calha entre os "justos"
    tabela={}
    for i in range(len(jogadores)-1):
        jog1=jogadores[i]
        for j in range(i+1,len(jogadores)):
            jog2=jogadores[j]
            _,_,_,tabelaX = jogaNpares(jogo,n,jog1,jog2)
            incorpora(tabela,tabelaX)
    return tabela

Criemos o El Caos Inteligente mas com uma visão a nível 3. 

In [17]:
el_caos_int3=JogadorAlfaBeta("El Caos Inteligente 3",3,f_caos_intel)

Façamos um torneio com os 3 jogadores, para o kalah justo nº 100 bem como 100 pares de jogos entre eles

In [18]:
torneio(100,[el_caos,el_caos_int6,el_caos_int3],100)

{'El Caos': 474, 'El Caos Inteligente 6': 667, 'El Caos Inteligente 3': 633}

### <span style="color:blue"> ***Chapiteau***</span>
<img src="Imagens\chapiteau_superman.gif" alt="Drawing" style="width: 150px;"/>

Deixamos que implementem o jogador <span style="color:blue"> ***Chapiteau***. </span> pois já o descrevemos de forma límpida. O gif com o super-homem é apenas para assustar porque na verdade <span style="color:blue"> ***Chapiteau***. </span> não é nada super.

## As funções de Avaliação
Terão de criar uma função de avaliação que recebe como input uma instância de `KalahState` e do jogador (0 se for SUL ou 1 se for NORTE).

### Nomes uniformes para os jogadores:
Para fazermos o torneio entre todos, é preciso uniformizar os nomes dos jogadores.

Todas as funções de avaliação e especialmente as seleccionadas para serem utilizadas no torneio terão que ter nomes uniformes, com a assinatura do grupo (XX substituído pelo número do grupo).

***func_XX(estado,jogador)***: que devolve o valor estimado do `estado` na perspectiva do `jogador` do grupo XX. 

## Campeonato
No torneio que vamos realizar, em formato de campeonato, cada um dos jogadores jogará contra cada um de todos os outros um nº $N$ de jogos como SUL, para pelo menos 4 limites diferentes de profundidade para o alfabeta, metade deles de ordem par e a outra metade de ordem ímpar.
Não podemos esquecer que além dos jogadores dos vários grupos, teremos a participação do <span style="color:blue"> ***Chapiteau***</span>.


### Pontuação de cada grupo
A pontuação final do campeonato é calculada com base na pontuação final de cada grupo, que corresponde ao

**somatório da pontuação obtida em todos os jogos em que a vitória vale 3, o empate vale 1 e a derrota 0**.


***Timeout:*** No caso de um dos jogadores ultrapassar o tempo limite para uma jogada será desclassificado. Usaremos bom senso para definir o que é o tempo que consideraremos como limite. Principalmente é fundamental não atrapalhar a execução do torneio múltiplo.

## Código a não ser alterado
<span style="color:red"> Não alterem</span> `jogos.py` nem `utils.py` e **não os devem submeter!** Iremos correr os mesmos 2 ficheiros para todos.
<br><span style="color:red">Não redefinam</span> funções com o mesmo nome das já existentes em `jogos.py` e `utils.py`
<br>O ideal é que <span style="color:red"> todas as vossas funções devem estar etiquetadas com a assinatura do grupo: o sufixo _XX, para que não se partilhe nem se sobreponha código.</span>

## Avaliação
A nota do vosso projecto depende apenas da pontuação final do torneio múltiplo.
   
**Torneio**: No torneio, em formato campeonato, como já dissermos, vamos incluir o nosso jogador <span style="color:blue"> ***Chapiteau***</span>,  Quem tiver a mesma pontuação total do torneio que o <span style="color:blue"> ***Chapiteau***</span> terá 10 valores. Quem tiver um score total inferior ao nosso jogador terá, naturalmente, negativa (nota menor do que 10) nesta componente. A nota será resultado da aplicação de uma função linear baseada no rank no campenato, tanto acima do <span style="color:blue"> ***Chapiteau***</span> como para baixo. O grupo que ficar em primeiro lugar acima do <span style="color:blue"> ***Chapiteau***</span> terá 20 valores.

**IMPORTANTE**: Qualquer função de avaliação que seja um clone do <span style="color:blue"> ***Chapiteau***</span> será desclassificado e ficará com nota 0.

### Entrega
Devem entregar um ficheiro ***IIA2324-proj2-jog-XX.py***, com o código com a função pedida e que vamos executar no torneio (incluindo eventuais funções auxiliares).

## Prazo

<img src="Imagens/chessPiscina.jpg" alt="Drawing" style="width: 250px;"/>

Até ao dia **14 de Novembro** às 23:59

### Penalizações
    - Entregue nas 3hs seguintes: sem penalização
    - A partir das 3h e até às 12h do dia seguinte: 20% de penalização
    - Não é aceite a partir das 12h do dia seguinte.